In [1]:
import pandas as pd
import matplotlib as mpl
import platform
import warnings
import numpy as np
import re

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# 한글 깨짐 방지 코드
if platform.system() == 'Windows':
    # Windows의 경우 'Malgun Gothic'을 많이 사용합니다.
    mpl.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':
    # macOS의 경우 'AppleGothic'을 사용하거나, 설치된 한글 폰트를 선택합니다.
    mpl.rcParams['font.family'] = 'AppleGothic'
else:
    # Linux의 경우 'NanumGothic' 등 한글 지원 폰트를 사용할 수 있습니다.
    mpl.rcParams['font.family'] = 'NanumGothic'

In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

### 1 info() 결과
1.1 train info() 결과
- primary key = ID
- 범주형 변수 : 국가, 분야, 투자단계, 인수여부, 상장여부, 기업가치(백억원)
- null값 존재 변수 : 분야, 직원 수, 고객수(백만명), 기업가치(백억원)
- target 변수 : 성공확률
- 모델 돌릴때 null값 처리하고 범주형 변수 처리하고 ID, 성공확률 컬럼 제거
### 
1.2 test info() 결과
- primary key = ID
- null값 존재 컬럼 : 분야, 직원 수, 고객수(백만명), 기업가치(백억원)
- 범주형 변수 : 국가, 분야, 투자단계, 인수여부, 상장여부, 기업가치(백억원)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4376 non-null   object 
 1   설립연도            4376 non-null   int64  
 2   국가              4376 non-null   object 
 3   분야              3519 non-null   object 
 4   투자단계            4376 non-null   object 
 5   직원 수            4202 non-null   float64
 6   인수여부            4376 non-null   object 
 7   상장여부            4376 non-null   object 
 8   고객수(백만명)        3056 non-null   float64
 9   총 투자금(억원)       4376 non-null   float64
 10  연매출(억원)         4376 non-null   float64
 11  SNS 팔로워 수(백만명)  4376 non-null   float64
 12  기업가치(백억원)       3156 non-null   object 
 13  성공확률            4376 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 478.8+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1755 entries, 0 to 1754
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              1755 non-null   object 
 1   설립연도            1755 non-null   int64  
 2   국가              1755 non-null   object 
 3   분야              1401 non-null   object 
 4   투자단계            1755 non-null   object 
 5   직원 수            1679 non-null   float64
 6   인수여부            1755 non-null   object 
 7   상장여부            1755 non-null   object 
 8   고객수(백만명)        1208 non-null   float64
 9   총 투자금(억원)       1755 non-null   float64
 10  연매출(억원)         1755 non-null   float64
 11  SNS 팔로워 수(백만명)  1755 non-null   float64
 12  기업가치(백억원)       1268 non-null   object 
dtypes: float64(5), int64(1), object(7)
memory usage: 178.4+ KB


### 2. 결측값 처리

In [400]:
# train[분야] null 값 처리
train['분야'].fillna(train['분야'].mode()[0], inplace=True)
# test[분야] null 값 처리
test['분야'].fillna(test['분야'].mode()[0], inplace=True)

In [401]:
# train[고객수(백만명)] null 값 처리
# 그룹 평균 보간: 투자단계 + 분야 기준
train['고객수(백만명)'] = train.groupby(['투자단계', '분야'])['고객수(백만명)'].transform(
    lambda x: x.fillna(x.mean())
)

# 혹시 그룹 평균이 NaN인 그룹이 있다면, 전체 평균으로 한 번 더 보간
train['고객수(백만명)'].fillna(train['고객수(백만명)'].mean(), inplace=True)

# test[고객수(백만명)] null 값 처리
# 그룹 평균 보간: 투자단계 + 분야 기준
test['고객수(백만명)'] = test.groupby(['투자단계', '분야'])['고객수(백만명)'].transform(
    lambda x: x.fillna(x.mean())
)

# 혹시 그룹 평균이 NaN인 그룹이 있다면, 전체 평균으로 한 번 더 보간
test['고객수(백만명)'].fillna(test['고객수(백만명)'].mean(), inplace=True)

In [ ]:
# train[직원 수] null 값 처리 : 평균값으로 처리, 나중에 결과 안좋을시 1인기업으로 생각해서 0으로 처리
train['직원 수'].fillna(0, inplace=True)
# test[직원 수] null 값 처리 : 평균값으로 처리, 나중에 결과 안좋을시 1인기업으로 생각해서 0으로 처리
test['직원 수'].fillna(0, inplace=True)

In [9]:
# 기업가치(백억원) 컬럼 null값 처리위해 숫자형 변수로 처리
def parse_기업가치(value):
    if pd.isnull(value):
        return np.nan
    elif '이상' in value:
        # "6000이상" -> 숫자만 추출해서 +500
        num = re.findall(r'\d+', value)
        return float(num[0]) + 500 if num else np.nan
    elif '-' in value:
        # "1500-2500" -> 평균값
        num = re.findall(r'\d+', value)
        if len(num) == 2:
            return (float(num[0]) + float(num[1])) / 2
    return np.nan

train['기업가치(백억원)'] = train['기업가치(백억원)'].apply(parse_기업가치)
test['기업가치(백억원)'] = test['기업가치(백억원)'].apply(parse_기업가치)

In [10]:
# train[기업가치(백억원)] null 값 처리
# 1. 투자단계 + 분야 기준 그룹 평균으로 보간
train['기업가치(백억원)'] = train.groupby(['투자단계', '분야'])['기업가치(백억원)'].transform(
    lambda x: x.fillna(x.mean())
)
# 2. 남은 결측치는 전체 중앙값으로 보간
train['기업가치(백억원)'].fillna(train['기업가치(백억원)'].median(), inplace=True)

# test[기업가치(백억원)] null 값 처리
# 1. 투자단계 + 분야 기준 그룹 평균으로 보간
test['기업가치(백억원)'] = test.groupby(['투자단계', '분야'])['기업가치(백억원)'].transform(
    lambda x: x.fillna(x.mean())
)
# 2. 남은 결측치는 전체 중앙값으로 보간
test['기업가치(백억원)'].fillna(test['기업가치(백억원)'].median(), inplace=True)

### 3. 파생변수 생각해보기

In [11]:
# 설립연차
train['설립연차'] = 2025 - train['설립연도']
test['설립연차'] = 2025 - test['설립연도']

In [12]:
# 기업 성장 속도
train['기업가치_연차비'] = train['기업가치(백억원)'] / train['설립연차']
test['기업가치_연차비'] = test['기업가치(백억원)'] / test['설립연차']

In [13]:
# 투자 효율성
train['매출_투자비'] = train['연매출(억원)'] / train['총 투자금(억원)']
test['매출_투자비'] = test['연매출(억원)'] / test['총 투자금(억원)']

In [14]:
# 브랜드 인지도 대비 실제 고객 전환율
train['팔로워_고객비'] = train['SNS 팔로워 수(백만명)'] / train['고객수(백만명)']
test['팔로워_고객비'] = test['SNS 팔로워 수(백만명)'] / test['고객수(백만명)']

In [15]:
# 이익률 = (연매출 - 총 투자금) / 연매출출
train['이익률'] = (train['연매출(억원)'] - train['총 투자금(억원)']) / train['연매출(억원)']
test['이익률'] = (test['연매출(억원)'] - test['총 투자금(억원)']) / test['연매출(억원)']

In [16]:
# 매출 성장률 = (연매출 - 이전 연매출) / 이전 연매출
train['매출_성장률'] = train['연매출(억원)'].pct_change().fillna(0)
test['매출_성장률'] = test['연매출(억원)'].pct_change().fillna(0)

In [17]:
# 총 자본 비율 = (총 투자금) / (연매출)
train['총_자본_비율'] = train['총 투자금(억원)'] / train['연매출(억원)']
test['총_자본_비율'] = test['총 투자금(억원)'] / test['연매출(억원)']

In [18]:
# 직원당_매출 = 연매출(억원) / 직원 수
train['직원당_매출'] = train['연매출(억원)'] / train['직원 수']
test['직원당_매출'] = test['연매출(억원)'] / test['직원 수']

In [20]:
train.isna().sum()

ID                   0
설립연도                 0
국가                   0
분야                 857
투자단계                 0
직원 수               174
인수여부                 0
상장여부                 0
고객수(백만명)          1320
총 투자금(억원)            0
연매출(억원)              0
SNS 팔로워 수(백만명)       0
기업가치(백억원)            0
성공확률                 0
설립연차                 0
기업가치_연차비             0
매출_투자비               0
팔로워_고객비           1320
이익률                  0
매출_성장률               0
총_자본_비율              0
직원당_매출             174
dtype: int64

### 4. 이상치 탐지
- 사분위수 이상치 탐지 결과: 기업가치_연차비, 매출_투자비, 팔로워_고객비 등 파생변수에서만 이상치 발생
- 파생변수 이지만 향후 모델에 사용할 수 있으므로 정제 필요하다고 생각 -> 클리핑 요소 사용
- 클리핑 : 값이 일정 범위를 벗어나면 그 범위의 최대/최소값으로 잘라내는 방법

In [412]:
# IQR 방식
def detect_outliers_iqr(df, columns):
    outlier_info = {}
    
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower) | (df[col] > upper)]
        outlier_info[col] = {
            'lower_bound': lower,
            'upper_bound': upper,
            'num_outliers': outliers.shape[0]
        }

    return outlier_info

In [413]:
# 수치형 컬럼만 적용
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns.tolist()
iqr_outliers = detect_outliers_iqr(train, numeric_cols)

# 결과 확인
for col, info in iqr_outliers.items():
    print(f"{col} → 이상치 수: {info['num_outliers']} (범위: {info['lower_bound']:.2f} ~ {info['upper_bound']:.2f})")


설립연도 → 이상치 수: 0 (범위: 1988.00 ~ 2036.00)
직원 수 → 이상치 수: 0 (범위: -2841.00 ~ 7585.00)
고객수(백만명) → 이상치 수: 0 (범위: -0.50 ~ 99.50)
총 투자금(억원) → 이상치 수: 0 (범위: -3233.62 ~ 9793.38)
연매출(억원) → 이상치 수: 0 (범위: -6803.12 ~ 19589.88)
SNS 팔로워 수(백만명) → 이상치 수: 0 (범위: -2.70 ~ 8.02)
기업가치(백억원) → 이상치 수: 0 (범위: -375.00 ~ 8625.00)
성공확률 → 이상치 수: 0 (범위: -0.05 ~ 1.15)
설립연차 → 이상치 수: 0 (범위: -11.00 ~ 37.00)
기업가치_연차비 → 이상치 수: 396 (범위: -330.83 ~ 1112.78)
매출_투자비 → 이상치 수: 512 (범위: -3.10 ~ 7.80)
팔로워_고객비 → 이상치 수: 381 (범위: -0.06 ~ 0.18)
이익률 → 이상치 수: 530 (범위: -1.13 ~ 1.85)
매출_성장률 → 이상치 수: 505 (범위: -2.84 ~ 3.37)
총_자본_비율 → 이상치 수: 530 (범위: -0.85 ~ 2.13)


In [414]:
# 수치형 컬럼만 적용
numeric_cols = test.select_dtypes(include=['float64', 'int64']).columns.tolist()
iqr_outliers = detect_outliers_iqr(test, numeric_cols)

# 결과 확인
for col, info in iqr_outliers.items():
    print(f"{col} → 이상치 수: {info['num_outliers']} (범위: {info['lower_bound']:.2f} ~ {info['upper_bound']:.2f})")

설립연도 → 이상치 수: 0 (범위: 1988.00 ~ 2036.00)
직원 수 → 이상치 수: 0 (범위: -2868.00 ~ 7616.00)
고객수(백만명) → 이상치 수: 0 (범위: -0.50 ~ 99.50)
총 투자금(억원) → 이상치 수: 0 (범위: -3099.50 ~ 9524.50)
연매출(억원) → 이상치 수: 0 (범위: -6550.50 ~ 19489.50)
SNS 팔로워 수(백만명) → 이상치 수: 0 (범위: -2.84 ~ 8.10)
기업가치(백억원) → 이상치 수: 0 (범위: -375.00 ~ 8625.00)
설립연차 → 이상치 수: 0 (범위: -11.00 ~ 37.00)
기업가치_연차비 → 이상치 수: 140 (범위: -344.45 ~ 1120.96)
매출_투자비 → 이상치 수: 205 (범위: -3.27 ~ 8.25)
팔로워_고객비 → 이상치 수: 148 (범위: -0.05 ~ 0.16)
이익률 → 이상치 수: 227 (범위: -0.99 ~ 1.79)
매출_성장률 → 이상치 수: 209 (범위: -2.92 ~ 3.52)
총_자본_비율 → 이상치 수: 227 (범위: -0.79 ~ 1.99)


In [415]:
# 클리핑
def clip_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower, upper)

In [416]:
# 클리핑 적용
for col in ['기업가치_연차비', '매출_투자비', '팔로워_고객비']:
    clip_outliers_iqr(train, col)
for col in ['기업가치_연차비', '매출_투자비', '팔로워_고객비']:
    clip_outliers_iqr(test, col)

### 5. 인코딩
- 변수 : 국가, 분야, 투자단계, 인수여부, 상장여부
- One-Hot Encoding (카테고리가 적을 때)
- Label Encoding (카테고리가 많거나 순서가 있을 때)

In [417]:
# one-hot 인코딩을 쓸지 label 인코딩을 쓸지 카테고리 수 파악
# 확인할 컬럼 리스트
cat_cols = ['국가', '분야', '투자단계', '인수여부', '상장여부']

# 각 컬럼의 카테고리 수 출력
for col in cat_cols:
    n_unique = train[col].nunique(dropna=True)
    print(f"'{col}' 카테고리 수: {n_unique}")
    print(f"→ 카테고리 목록: {train[col].dropna().unique()}\n")

'국가' 카테고리 수: 10
→ 카테고리 목록: ['CT005' 'CT006' 'CT007' 'CT002' 'CT008' 'CT010' 'CT001' 'CT009' 'CT003'
 'CT004']

'분야' 카테고리 수: 10
→ 카테고리 목록: ['이커머스' '핀테크' '기술' '에너지' '에듀테크' '게임' '헬스케어' '물류' '푸드테크' 'AI']

'투자단계' 카테고리 수: 5
→ 카테고리 목록: ['Series A' 'Seed' 'Series C' 'Series B' 'IPO']

'인수여부' 카테고리 수: 2
→ 카테고리 목록: ['No' 'Yes']

'상장여부' 카테고리 수: 2
→ 카테고리 목록: ['No' 'Yes']



In [418]:
# 카테고리 수가 10개 이하이므로 one-hot 인코딩으로 선택
train = pd.get_dummies(train, columns=cat_cols, drop_first=False, dtype=int)
test = pd.get_dummies(test, columns=cat_cols, drop_first=False, dtype=int)
# 컬럼 불일치 일시
missing_cols = set(train.columns) - set(test.columns)
missing_cols.discard('성공확률')  # test에 없는 타겟컬럼은 제외
for col in missing_cols:
    test[col] = 0

### 6. 스케일링
- StandardScaler	평균 0, 표준편차 1로 변환. 정규분포에 가까운 경우 적합.
- MinMaxScaler	0~1 범위로 정규화. 데이터 분포에 관계없이 모든 범위 고정. 이상치에 민감함.
- RobustScaler	중앙값(median)과 IQR(4분위 범위) 기준으로 스케일링. 이상치에 강함.
- MaxAbsScaler	절댓값 기준으로 -1~1로 조정. 희소행렬(sparse matrix) 에 적합.

In [419]:
# 스케일링 방식을 비교해봤을 때 스케일링 대상 컬럼들 간의 이상치 영향이 크지 않거나, 선형 회귀 모델에 미치는 영향이 제한적이라 판단
# 이 경우 StandardScaler를 사용하는 것이 무난하다고 생각이 들어 StandardScaler 사용
from sklearn.preprocessing import StandardScaler

# 스케일링할 수치형 컬럼 리스트
numeric_cols = ['직원 수', '고객수(백만명)', '총 투자금(억원)', 
                '연매출(억원)', 'SNS 팔로워 수(백만명)', '기업가치(백억원)', 
                '설립연차', '기업가치_연차비', '매출_투자비', '팔로워_고객비', '이익률', '매출_성장률', '총_자본_비율']

# 스케일러 학습 및 변환
scaler = StandardScaler()
train[numeric_cols] = scaler.fit_transform(train[numeric_cols])

# test 데이터도 동일한 스케일러로 변환
test[numeric_cols] = scaler.transform(test[numeric_cols])

In [420]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 45 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4376 non-null   object 
 1   설립연도            4376 non-null   int64  
 2   직원 수            4376 non-null   float64
 3   고객수(백만명)        4376 non-null   float64
 4   총 투자금(억원)       4376 non-null   float64
 5   연매출(억원)         4376 non-null   float64
 6   SNS 팔로워 수(백만명)  4376 non-null   float64
 7   기업가치(백억원)       4376 non-null   float64
 8   성공확률            4376 non-null   float64
 9   설립연차            4376 non-null   float64
 10  기업가치_연차비        4376 non-null   float64
 11  매출_투자비          4376 non-null   float64
 12  팔로워_고객비         4376 non-null   float64
 13  이익률             4376 non-null   float64
 14  매출_성장률          4376 non-null   float64
 15  총_자본_비율         4376 non-null   float64
 16  국가_CT001        4376 non-null   int32  
 17  국가_CT002        4376 non-null   i

### 7. 상관관계 분석
- 상관관계 분석 결과 설립연도, 설립연차, 기업가치_연차비, 총 투자금(억원), 매출_투자비, 고객수(백만명), 팔로워_고객비, SNS 팔로워 수(백만명), 팔로워_고객비, 연매출(억원), 기업가치(백억원), 기업가치_연차비가 연관관계가 있음을 확인

In [421]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 상관계수 행렬 (숫자형 변수만)
corr_matrix = train.corr(numeric_only=True)

# 상관계수 행렬을 길게 풀어서 상관관계 쌍으로 만들기
corr_pairs = corr_matrix.unstack()

# 자기 자신과의 상관관계(1.0) 제거
corr_pairs = corr_pairs[corr_pairs.index.get_level_values(0) != corr_pairs.index.get_level_values(1)]

# 중복 제거 (ex. A-B와 B-A는 하나로 취급)
corr_pairs = corr_pairs.drop_duplicates()

# 절대값 기준으로 정렬
sorted_corr_pairs = corr_pairs.reindex(corr_pairs.abs().sort_values(ascending=False).index)

# 상위 10개 확인 (원하면 숫자 바꿔도 됩니다)
print("상관관계 높은 피처 쌍 (자기자신 제외):")
print(sorted_corr_pairs.head(10))

plt.show()

상관관계 높은 피처 쌍 (자기자신 제외):
설립연도            설립연차       -1.000000
인수여부_No         인수여부_Yes   -1.000000
이익률             총_자본_비율    -1.000000
상장여부_No         상장여부_Yes   -1.000000
설립연도            기업가치_연차비    0.827011
설립연차            기업가치_연차비   -0.827011
총 투자금(억원)       매출_투자비     -0.709510
고객수(백만명)        팔로워_고객비    -0.644710
SNS 팔로워 수(백만명)  팔로워_고객비     0.607358
연매출(억원)         매출_투자비      0.504395
dtype: float64


In [422]:
import pandas as pd

# 숫자형 피처에 대한 상관계수 행렬 계산
corr_matrix = train.corr(numeric_only=True)

# '성공확률'과의 상관계수만 추출
target = '성공확률'
target_corr = corr_matrix[target].drop(target)  # 자기 자신 제외

# 절대값 기준 정렬
target_corr_sorted = target_corr.reindex(target_corr.abs().sort_values(ascending=False).index)

# 출력: 높은 상관 피처
print("✅ 성공확률과 상관관계가 높은 피처:")
print(target_corr_sorted)

# 기준 설정
threshold_high = 0.01
threshold_low = 0.01

# 높은 상관계수 피처
print(f"\n✅ 상관계수 절대값이 {threshold_high} 이상인 피처:")
print(target_corr_sorted[abs(target_corr_sorted) > threshold_high])

# 낮은 상관계수 피처
print(f"\n⚠️ 상관계수 절대값이 {threshold_low} 이하인 피처:")
print(target_corr_sorted[abs(target_corr_sorted) <= threshold_low])


✅ 성공확률과 상관관계가 높은 피처:
분야_헬스케어          -0.050412
기업가치(백억원)         0.038361
분야_핀테크            0.036031
상장여부_Yes          0.031413
상장여부_No          -0.031413
국가_CT002         -0.030246
팔로워_고객비           0.028075
국가_CT009          0.024956
분야_이커머스          -0.019987
분야_에너지            0.019899
투자단계_Series B    -0.019373
연매출(억원)          -0.018994
투자단계_Seed         0.018269
SNS 팔로워 수(백만명)    0.017927
국가_CT008          0.017290
기업가치_연차비          0.015176
총 투자금(억원)        -0.014939
국가_CT006          0.014471
고객수(백만명)         -0.014135
직원 수             -0.013776
인수여부_No           0.012065
인수여부_Yes         -0.012065
매출_성장률           -0.011403
국가_CT004         -0.010975
국가_CT001         -0.008728
이익률              -0.006569
총_자본_비율           0.006569
국가_CT010         -0.005634
분야_게임            -0.005531
분야_푸드테크          -0.004299
국가_CT007          0.003733
분야_기술             0.003571
국가_CT003         -0.003263
분야_물류             0.003061
투자단계_IPO         -0.002592
투자단계_Series C     0.002185
설립연차   

### 8. 모델링

In [423]:
# target과 제외할 칼럼
target_col = '성공확률'
drop_cols = ['ID', '매출_투자비', '총_자본_비율', '설립연도', target_col]

# 학습용 피처, 타겟 분리
X = train.drop(columns=drop_cols)
y = train[target_col]

In [424]:
from sklearn.model_selection import train_test_split
# 훈련/검증 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [425]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

models = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1),
    'RandomForest': RandomForestRegressor(max_depth=None, 
    max_features='sqrt', 
    min_samples_leaf=1, 
    min_samples_split=2, 
    n_estimators=300,
    random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'LightGBM': LGBMRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(verbose=0, random_state=42)
}

results = []

# 가중치 정의
weights_valid = np.where(y_valid < 0.5, 2, 1)

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    wmae = np.sum(weights_valid * np.abs(y_valid - pred)) / np.sum(weights_valid)
    results.append((name, wmae))

# 결과 정렬
results.sort(key=lambda x: x[1])
print("📊 Weighted MAE 비교:")
for name, wmae in results:
    print(f"{name:<15} | WMAE: {wmae:.4f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000316 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2300
[LightGBM] [Info] Number of data points in the train set: 3500, number of used features: 40
[LightGBM] [Info] Start training from score 0.534486
📊 Weighted MAE 비교:
RandomForest    | WMAE: 0.2147
CatBoost        | WMAE: 0.2169
LightGBM        | WMAE: 0.2170
Ridge           | WMAE: 0.2175
Linear          | WMAE: 0.2175
Lasso           | WMAE: 0.2178
GradientBoosting | WMAE: 0.2181
XGBoost         | WMAE: 0.2219


In [426]:
sample_submission = pd.read_csv('data/sample_submission.csv')  # 파일 경로 확인 필요

# ✅ 최종 모델: RandomForest 사용하여 test 데이터 예측 및 submission 저장
final_model = RandomForestRegressor(
    max_depth=None, 
    max_features='sqrt', 
    min_samples_leaf=1, 
    min_samples_split=2, 
    n_estimators=300,
    random_state=42
)
final_model.fit(X, y)  # 전체 학습 데이터로 다시 학습

X_test = test.drop(columns=['ID', '매출_투자비', '총_자본_비율', '설립연도'])
final_predictions = final_model.predict(X_test)

# 예측 결과 저장
sample_submission['성공확률'] = final_predictions
sample_submission.to_csv('data/baseline_submission.csv', index=False, encoding='utf-8-sig')
print("✅ submission 파일이 저장되었습니다: baseline_submission.csv")

✅ submission 파일이 저장되었습니다: baseline_submission.csv


In [102]:
from sklearn.model_selection import GridSearchCV

# 파라미터 그리드 정의
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# GridSearchCV를 사용하여 최적 파라미터 찾기
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

# 최적의 파라미터
print("Best parameters:", grid_search.best_params_)

# 최적 모델을 사용하여 예측
final_model = grid_search.best_estimator_
final_model.fit(X_train, y_train)
final_predictions = final_model.predict(X_test)


Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [103]:
import pandas as pd

# 특성 중요도 출력
feature_importance = pd.Series(final_model.feature_importances_, index=X.columns).sort_values(ascending=False)
print(feature_importance)

# 중요하지 않은 특성 제거
important_features = feature_importance[feature_importance > 0.01].index  # 중요도가 0.01 이상인 특성만 선택
X = X[important_features]


직원 수              0.069322
매출_성장률            0.067265
총 투자금(억원)         0.066659
연매출(억원)           0.065939
팔로워_고객비           0.060790
고객수(백만명)          0.059241
이익률               0.058480
총_자본_비율           0.058374
기업가치_연차비          0.055821
매출_투자비            0.053077
SNS 팔로워 수(백만명)    0.052088
설립연차              0.041210
설립연도              0.040952
기업가치(백억원)         0.039641
상장여부_No           0.009188
인수여부_Yes          0.009122
인수여부_No           0.009121
상장여부_Yes          0.009120
투자단계_IPO          0.009065
투자단계_Series A     0.009032
투자단계_Seed         0.008941
분야_에너지            0.008755
투자단계_Series B     0.008630
투자단계_Series C     0.008105
국가_CT010          0.007228
국가_CT004          0.007045
국가_CT008          0.006963
분야_헬스케어           0.006945
국가_CT001          0.006944
국가_CT006          0.006818
국가_CT009          0.006786
국가_CT007          0.006659
국가_CT002          0.006645
분야_게임             0.006587
국가_CT005          0.006428
국가_CT003          0.006377
분야_이커머스           0.006374
분

In [ ]:
# 인수와 상장을 합치는게 좋아보임, 투자단계랑 분야를 파 볼 필요가 있을듯